## Import thư viện quan trọng.

In [ ]:
import warnings
from sklearn.exceptions import ConvergenceWarning

warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=ConvergenceWarning)
warnings.filterwarnings("ignore", category=RuntimeWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore")

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sbn
import matplotlib.pyplot as plt
from collections import Counter
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import fpgrowth, association_rules

## Upload dataset:
1. Review
2. Sách

## Tiền xử lý

In [ ]:
! gdown 1oo0dYlJFJoHDlBMmW5xV18Xx81r7b7gI

Downloading...
From (original): https://drive.google.com/uc?id=1oo0dYlJFJoHDlBMmW5xV18Xx81r7b7gI
From (redirected): https://drive.google.com/uc?id=1oo0dYlJFJoHDlBMmW5xV18Xx81r7b7gI&confirm=t&uuid=f9a02869-5250-4706-bdff-3144a0836b9f
To: /content/goodreads_reviews.csv
100% 1.38G/1.38G [00:20<00:00, 66.8MB/s]


In [ ]:
! gdown 18idPyOXCIYrksANBJrYNVt4Hgth0GDKv

Downloading...
From: https://drive.google.com/uc?id=18idPyOXCIYrksANBJrYNVt4Hgth0GDKv
To: /content/goodreads_works.csv
100% 16.2M/16.2M [00:00<00:00, 137MB/s]


In [ ]:
df_review = pd.read_csv('/content/goodreads_reviews.csv')
df_review1 = pd.read_csv('/content/goodreads_reviews.csv')

In [ ]:
df_review.sample(10)

,review_id,user_id,work_id,started_at,read_at,date_added,rating,review_text,n_votes,n_comments
920307,34ef9eb339c90b10f318a796525699f4,7f2ae37c774731ea59d515e9843ca280,49672494,2017-07-24 00:00:00.000,2017-07-25 00:00:00.000,2017-01-16 00:00:00.000,4.0,"Tash Hearts Tolstoy was an enjoyable, but pred...",5,0
38348,0c39c4df6b84e63a957b1c770edadb98,65406e4ce07a1e1cb0037767a2d1c961,14681975,NaN,NaN,2013-04-11 00:00:00.000,3.0,** spoiler alert ** \n Negatives: swearing (th...,0,0
286843,af9d58169fd5d21b90d07e2d779bbb37,3e5103883b8671f16be0264b988c3bbf,22277074,2014-10-11 00:00:00.000,2014-10-12 00:00:00.000,2014-10-11 00:00:00.000,3.0,"3.5 stars. Has some awkward ""first novel"" mome...",0,0
421097,e83cedc1c761c1c53de73294eae85f00,3ef076fcdb4bd84ee2aa5570623477a2,7329307,2013-12-01 00:00:00.000,2013-12-11 00:00:00.000,2011-05-12 00:00:00.000,5.0,I really liked this book. This is the same aut...,0,0
804130,283a2911199cbd2f93052841e8e76f07,dabc2df3003ef6cc075033bd525dcf41,1499998,NaN,NaN,2009-06-23 00:00:00.000,4.0,It's hard to say much about this book without ...,1,0
409162,8414f8253adfd4e09c4118904a41e25d,0809bab1d157392f4cc08b78f3608744,43145292,2015-02-14 00:00:00.000,2015-02-15 00:00:00.000,2015-12-31 00:00:00.000,5.0,MY ALL TIME FAVORITE READ OF 2015 \n Re-read J...,0,0
149588,d74741c991f11023196c7a3d87593629,ae1bea88010d584ec9e07ca81ff1f336,819610,NaN,2014-07-19 00:00:00.000,2014-07-21 00:00:00.000,3.0,This sequel to the Curse of Chalion sort of pi...,0,0
216648,10893077acb765ad4d95d20175b892a9,1f08a2c675b5f860cc3bdad769a93f48,3821158,NaN,2011-08-01 00:00:00.000,2012-06-09 00:00:00.000,4.0,** spoiler alert ** \n A is back. What? How? M...,0,0
894482,55306893f3e98342c51c733d1bdfa2eb,235a3175a1ed3bd98b81e4460d09d6fb,16827462,NaN,2014-05-28 00:00:00.000,2013-02-02 00:00:00.000,5.0,I cried even more the second time around. Than...,3,0
202383,de47600fd9251a94196e112fd9d7c061,500d451862af597ef57b3b85817912b9,21539506,NaN,2016-10-10 00:00:00.000,2012-01-03 00:00:00.000,4.0,This is the first time I've given a Rick Riord...,0,0


In [ ]:
df_review.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1143887 entries, 0 to 1143886
Data columns (total 10 columns):
 #   Column       Non-Null Count    Dtype  
---  ------       --------------    -----  
 0   review_id    1143887 non-null  object 
 1   user_id      1143887 non-null  object 
 2   work_id      1143887 non-null  int64  
 3   started_at   796392 non-null   object 
 4   read_at      1031765 non-null  object 
 5   date_added   1143887 non-null  object 
 6   rating       1106569 non-null  float64
 7   review_text  1143887 non-null  object 
 8   n_votes      1143887 non-null  int64  
 9   n_comments   1143887 non-null  int64  
dtypes: float64(1), int64(3), object(6)
memory usage: 87.3+ MB


### Lọc review tốt & những users review trên 2 sách trở lên
Để đảm bảo làm luật kết hợp cho những quyển sách có giá trị và giỏ hàng của users (giỏ hàng reviews) phải có trên 2 sản phẩm sách

In [ ]:
df_review = df_review[df_review['rating'] > 3]

In [ ]:
print(len(df_review))

732968


In [ ]:
df_review.duplicated().sum()

np.int64(0)

In [ ]:
df_review.isna().sum()

,0
review_id,0
user_id,0
work_id,0
started_at,207017
read_at,55605
date_added,0
rating,0
review_text,0
n_votes,0
n_comments,0


Không có dòng review nào null, lọc ra chỉ những user_id có 2 review

In [ ]:
multi_rv_user = df_review.groupby('user_id')['work_id'].count().reset_index(name='work_count')
multi_rv_user = multi_rv_user[multi_rv_user['work_count'] >= 2]
multi_rv_user

,user_id,work_count
0,000883382802f2d95a3dd545bb953882,111
1,000a1016fda6008d1edbba720ca00851,39
3,0011e1a9112b3d798702ef5b20bbf35b,12
5,0019de4561419b7543238e0979f2f33e,46
6,00204424763e8233c5f53f0729f2304f,54
...,...,...
18379,fff7bfd82b89fa347edfe9a82ac0c61b,7
18380,fffc34d137f5c5c5e1ca1d6f325a4dcf,58
18381,fffce7dae5ac5e8fb6288d81658ececc,51
18382,fffe42f796fb60bd03468ac206c321c1,40


Lọc ra những user có trên 1 quyển sách được reviewed

In [ ]:
df_review = df_review[df_review['user_id'].isin(multi_rv_user['user_id'])]

In [ ]:
df_review.info()

<class 'pandas.core.frame.DataFrame'>
Index: 732186 entries, 0 to 1143598
Data columns (total 10 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   review_id    732186 non-null  object 
 1   user_id      732186 non-null  object 
 2   work_id      732186 non-null  int64  
 3   started_at   525465 non-null  object 
 4   read_at      676704 non-null  object 
 5   date_added   732186 non-null  object 
 6   rating       732186 non-null  float64
 7   review_text  732186 non-null  object 
 8   n_votes      732186 non-null  int64  
 9   n_comments   732186 non-null  int64  
dtypes: float64(1), int64(3), object(6)
memory usage: 61.4+ MB


In [ ]:
df_book = pd.read_csv('/content/goodreads_works.csv')

In [ ]:
df_book.head(5)

,work_id,isbn,isbn13,original_title,author,original_publication_year,num_pages,description,genres,image_url,reviews_count,text_reviews_count,5_star_ratings,4_star_ratings,3_star_ratings,2_star_ratings,1_star_ratings,ratings_count,avg_rating,similar_books
0,2919130,1416534601,9.781417e+12,Nocturnes,John Connolly,2004.0,NaN,NaN,"fiction, fantasy, paranormal, mystery, thrille...",https://s.gr-assets.com/assets/nophoto/book/11...,8820,338,1118,1601,1029,190,58,3996,3.9,NaN
1,52087333,NaN,NaN,Draw Play,Tia Lewis,2016.0,NaN,Jake:\nI can't believe my coach assigned me a ...,"romance, fiction",https://s.gr-assets.com/assets/nophoto/book/11...,2482,204,204,353,274,77,29,937,3.7,NaN
2,1649583,1416505520,9.781417e+12,Citizen of the Galaxy,Robert A. Heinlein,1957.0,NaN,"In a distant galaxy, the atrocity of slavery w...","fiction, young-adult, fantasy, paranormal, chi...",https://s.gr-assets.com/assets/nophoto/book/11...,16506,447,3539,4351,2863,444,53,11250,4.0,NaN
3,688299,0060541830,9.780061e+12,Congo,Michael Crichton,1980.0,NaN,"Deep in the African rain forest, near the lege...","fiction, mystery, thriller, crime, fantasy, pa...",https://s.gr-assets.com/assets/nophoto/book/11...,170916,1633,25081,45775,48505,14001,2926,136288,3.6,NaN
4,3464264,0451528824,9.780452e+12,Anne of Green Gables,L.M. Montgomery,1908.0,NaN,"Everyone's favorite redhead, the spunky Anne S...","fiction, young-adult, children, history, histo...",https://s.gr-assets.com/assets/nophoto/book/11...,743392,14586,272952,161856,81578,19933,9099,545418,4.2,NaN


### Bộ dữ liệu hoàn chỉnh df

In [ ]:
df = pd.merge(df_review[['user_id', 'work_id', 'review_id','review_text', 'rating', 'started_at','read_at']], df_book[['work_id', 'original_title', 'genres', 'author', 'original_publication_year','avg_rating']],
              on = 'work_id', how = 'left')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 732186 entries, 0 to 732185
Data columns (total 12 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   user_id                    732186 non-null  object 
 1   work_id                    732186 non-null  int64  
 2   review_id                  732186 non-null  object 
 3   review_text                732186 non-null  object 
 4   rating                     732186 non-null  float64
 5   started_at                 525465 non-null  object 
 6   read_at                    676704 non-null  object 
 7   original_title             732186 non-null  object 
 8   genres                     732186 non-null  object 
 9   author                     732186 non-null  object 
 10  original_publication_year  731996 non-null  float64
 11  avg_rating                 732186 non-null  float64
dtypes: float64(3), int64(1), object(8)
memory usage: 67.0+ MB


In [ ]:
df = df.dropna(subset=['genres'])

Tiền xử lý:
- thời gian bắt đầu và thời gian hoàn thành không được null
- thời gian bắt đầu phải trước thời gian hoàn thành cuốn sách

In [ ]:
df = df[df['started_at'].notnull() & df['read_at'].notnull()]
df = df[df['started_at'] < df['read_at']]

In [ ]:
df.head()

,user_id,work_id,review_id,review_text,rating,started_at,read_at,original_title,genres,author,original_publication_year,avg_rating
135566,631853097d378547c63cc2c72be75cd3,41711738,ada43c5045253d1fc7bea6be2596fce2,Real life changing.,5.0,2015-10-20 00:00:00.000,2016-04-21 00:00:00.000,Ren Sheng gatokimekuPian dukenoMo Fa,non-fiction,Marie Kondo,2011.0,3.8
135567,2f345f02ec439d1a25d8d319e9ca6748,18979356,683151c53648bd8a2404525093359e86,"Wow. Just, wow. I promise a real review soon, ...",5.0,2012-09-27 00:00:00.000,2012-10-19 00:00:00.000,Wool Omnibus,"fiction, fantasy, paranormal, mystery, thrille...",Hugh Howey,2012.0,4.2
135568,55475556ab32863fed66cb94a404af71,18979356,2a1253730a71cf698204bc6797ee0c81,I like this book better now that I've had time...,5.0,2012-09-13 00:00:00.000,2012-11-13 00:00:00.000,Wool Omnibus,"fiction, fantasy, paranormal, mystery, thrille...",Hugh Howey,2012.0,4.2
135569,c3a79107fe2555b144a250478faec8c8,18979356,94db91ef52109aeedb293b01fde17918,Great book! I was hooked after the first chapt...,5.0,2013-08-26 00:00:00.000,2013-09-04 00:00:00.000,Wool Omnibus,"fiction, fantasy, paranormal, mystery, thrille...",Hugh Howey,2012.0,4.2
135570,f9a3d21c44f40ac7a64e6a4b10b30c10,12946965,8d6421ac38182defa91db71f14c15580,"For whatever reason, I put off reading this bo...",5.0,2012-11-18 00:00:00.000,2012-11-23 00:00:00.000,"Unbroken: A World War II Story of Survival, Re...","non-fiction, history, historical fiction, biog...",Laura Hillenbrand,2010.0,4.4


## Luật kết hợp sách với sách
Những quyển sách nào thường được review chung với nhau

In [ ]:
df_raw = df.dropna(subset=['original_title'])

In [ ]:
df_raw = df_raw[['user_id', 'work_id', 'original_title']]

In [ ]:
df_raw.head()

,user_id,work_id,original_title
135566,631853097d378547c63cc2c72be75cd3,41711738,Ren Sheng gatokimekuPian dukenoMo Fa
135567,2f345f02ec439d1a25d8d319e9ca6748,18979356,Wool Omnibus
135568,55475556ab32863fed66cb94a404af71,18979356,Wool Omnibus
135569,c3a79107fe2555b144a250478faec8c8,18979356,Wool Omnibus
135570,f9a3d21c44f40ac7a64e6a4b10b30c10,12946965,"Unbroken: A World War II Story of Survival, Re..."


In [ ]:
transactions = (
    df_raw.groupby('user_id')['original_title']
          .apply(list)
          .tolist()
)

In [ ]:
# Đếm tần suất xuất hiện sách
all_items = [item for trans in transactions for item in trans]
item_freq = Counter(all_items)

# Giữ top N sách phổ biến
TOP_N_ITEMS = 300
popular_items = set([i for i, _ in item_freq.most_common(TOP_N_ITEMS)])

# Lọc transactions
transactions_filtered = [
    [item for item in trans if item in popular_items]
    for trans in transactions
]

# Bỏ transaction quá ngắn
transactions_filtered = [t for t in transactions_filtered if len(t) >= 2]

print("Transactions sau lọc:", len(transactions_filtered))

Transactions sau lọc: 11252


In [ ]:
te = TransactionEncoder()
te_ary = te.fit(transactions).transform(transactions)

df_onehot = pd.DataFrame(
    te_ary,
    columns=te.columns_
)

df_onehot.head()

,"""D"" is for Deadbeat","""M"" is for Malice","""Shouldn't You Be in School?""",#Junkie,#Nerd,#Player,#Poser,#Scandal,#Selfie,#famous,...,uzumaki 1,uzumaki 3,vN,vuanpaiaQi Shi 1,vuanpaiaQi Shi 2,vuanpaiaQi Shi 4,vuanpaiaQi Shi 7,vuanpaiaQi Shi 9,xxxHOLiC #1,yolo
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


### 2 items

In [ ]:
from mlxtend.frequent_patterns import fpgrowth

frequent_itemsets = fpgrowth(
    df_onehot,
    min_support=0.01,
    use_colnames=True,
    max_len=2
)

print("Số frequent itemsets:", len(frequent_itemsets))
frequent_itemsets.head()

Số frequent itemsets: 1233


,support,itemsets
0,0.054112,(Insurgent)
1,0.048025,(Clockwork Princess)
2,0.039016,(Clockwork Prince)
3,0.034512,(Allegiant)
4,0.032199,"(Unravel Me (Shatter Me, #2))"


In [ ]:
from mlxtend.frequent_patterns import association_rules

rules = association_rules(
    frequent_itemsets,
    metric="confidence",
    min_threshold=0.2
)

rules = rules.sort_values(
    by=["lift", "confidence"],
    ascending=False
)

rules.head(10)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
839,(The Battle of the Labyrinth),(The Titan's Curse),0.018139,0.018991,0.010226,0.563758,29.685854,1.0,0.009881,2.248775,0.984165,0.380090,0.555313,0.551110
840,(The Titan's Curse),(The Battle of the Labyrinth),0.018991,0.018139,0.010226,0.538462,29.685854,1.0,0.009881,2.127366,0.985020,0.380090,0.529935,0.551110
846,(The Golden Lily),(Bloodlines),0.017834,0.019478,0.010104,0.566553,29.087180,1.0,0.009757,2.262150,0.983154,0.371365,0.557943,0.542651
845,(Bloodlines),(The Golden Lily),0.019478,0.017834,0.010104,0.518750,29.087180,1.0,0.009757,2.040864,0.984802,0.371365,0.510011,0.542651
835,(The Lost Hero),(The Son of Neptune),0.020147,0.018930,0.010713,0.531722,28.088944,1.0,0.010331,2.095059,0.984228,0.377682,0.522687,0.548819
836,(The Son of Neptune),(The Lost Hero),0.018930,0.020147,0.010713,0.565916,28.088944,1.0,0.010331,2.257290,0.983007,0.377682,0.556991,0.548819
831,(The Sea of Monsters),(The Titan's Curse),0.021973,0.018991,0.011504,0.523546,27.568373,1.0,0.011087,2.058979,0.985379,0.390496,0.514322,0.564657
832,(The Titan's Curse),(The Sea of Monsters),0.018991,0.021973,0.011504,0.605769,27.568373,1.0,0.011087,2.480848,0.982383,0.390496,0.596912,0.564657
442,(Morning Star),(Golden Son),0.017834,0.025138,0.012356,0.692833,27.560653,1.0,0.011908,3.173716,0.981216,0.403579,0.684912,0.592179
441,(Golden Son),(Morning Star),0.025138,0.017834,0.012356,0.491525,27.560653,1.0,0.011908,1.931593,0.988567,0.403579,0.482292,0.592179


In [ ]:
rules_clean = rules[
    (rules["lift"] > 1.2) &
    (rules["confidence"] > 0.3)
]

rules_clean.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
839,(The Battle of the Labyrinth),(The Titan's Curse),0.018139,0.018991,0.010226,0.563758,29.685854,1.0,0.009881,2.248775,0.984165,0.380090,0.555313,0.551110
840,(The Titan's Curse),(The Battle of the Labyrinth),0.018991,0.018139,0.010226,0.538462,29.685854,1.0,0.009881,2.127366,0.985020,0.380090,0.529935,0.551110
846,(The Golden Lily),(Bloodlines),0.017834,0.019478,0.010104,0.566553,29.087180,1.0,0.009757,2.262150,0.983154,0.371365,0.557943,0.542651
845,(Bloodlines),(The Golden Lily),0.019478,0.017834,0.010104,0.518750,29.087180,1.0,0.009757,2.040864,0.984802,0.371365,0.510011,0.542651
835,(The Lost Hero),(The Son of Neptune),0.020147,0.018930,0.010713,0.531722,28.088944,1.0,0.010331,2.095059,0.984228,0.377682,0.522687,0.548819


### 3 items trở lên

In [ ]:
import pandas as pd
from collections import Counter
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import fpgrowth, association_rules


def preprocess_transactions(transactions, top_n=500, min_len=3):

    all_items = [item for trans in transactions for item in trans]
    item_freq = Counter(all_items)
    popular_items = set([i for i, _ in item_freq.most_common(top_n)])

    filtered_data = [
        [item for item in trans if item in popular_items]
        for trans in transactions
    ]
    # Chỉ giữ giao dịch có từ 3 món trở lên
    filtered_data = [t for t in filtered_data if len(t) >= min_len]

    print(f"Số lượng giao dịch (>=3 món) sau khi lọc: {len(filtered_data)}")
    return filtered_data


def build_complex_rules(transactions_filtered, min_support=0.005, max_len=4):

    te = TransactionEncoder()
    te_ary = te.fit(transactions_filtered).transform(transactions_filtered)
    df_onehot = pd.DataFrame(te_ary, columns=te.columns_)


    frequent_itemsets = fpgrowth(
        df_onehot,
        min_support=min_support,
        use_colnames=True,
        max_len=max_len
    )


    frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(len)


    rules = association_rules(
        frequent_itemsets,
        metric="lift",
        min_threshold=1.2
    )


    rules["total_items"] = rules["antecedents"].apply(len) + rules["consequents"].apply(len)


    rules = rules.sort_values(by=["lift", "confidence"], ascending=False)

    return rules


def get_bundle_recommendations(current_cart, rules_df, target_size=3, top_k=5):
    """
    Gợi ý sách dựa trên giỏ hàng hiện tại để tạo thành bộ target_size
    current_cart: list các sách đang có (ví dụ: ['Sách A', 'Sách B'])
    """
    cart_set = set(current_cart)


    mask = (
        rules_df['antecedents'].apply(lambda x: x.issubset(cart_set)) &
        (rules_df['total_items'] == target_size)
    )

    recommendations = rules_df[mask]
    return recommendations[['consequents', 'confidence', 'lift', 'total_items']].head(top_k)


tx_filtered = preprocess_transactions(transactions, top_n=800, min_len=3)


complex_rules = build_complex_rules(tx_filtered, min_support=0.02, max_len=4)


print("\n--- Top 5 luật cho bộ 3 món ---")
display(complex_rules[complex_rules['total_items'] == 3].head(5))

print("\n--- Top 5 luật cho bộ 4 món ---")
display(complex_rules[complex_rules['total_items'] == 4].head(5))



Số lượng giao dịch (>=3 món) sau khi lọc: 11660

--- Top 5 luật cho bộ 3 món ---


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski,total_items
322,"(The Dream Thieves, The Raven King)","(Blue Lily, Lily Blue)",0.030703,0.058491,0.023671,0.770950,13.180753,1.0,0.021875,4.110492,0.953404,0.361257,0.756720,0.587821,3
323,"(Blue Lily, Lily Blue)","(The Dream Thieves, The Raven King)",0.058491,0.030703,0.023671,0.404692,13.180753,1.0,0.021875,1.628228,0.981543,0.361257,0.385835,0.587821,3
309,"(The Raven Boys, The Raven King)","(Blue Lily, Lily Blue)",0.029931,0.058491,0.022298,0.744986,12.736852,1.0,0.020548,3.691986,0.949920,0.337224,0.729143,0.563109,3
312,"(Blue Lily, Lily Blue)","(The Raven Boys, The Raven King)",0.058491,0.029931,0.022298,0.381232,12.736852,1.0,0.020548,1.567741,0.978734,0.337224,0.362140,0.563109,3
294,"(The Raven Boys, Blue Lily, Lily Blue)",(The Dream Thieves),0.037479,0.066381,0.029503,0.787185,11.858632,1.0,0.027015,4.387006,0.951328,0.396770,0.772054,0.615815,3



--- Top 5 luật cho bộ 4 món ---


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski,total_items


## The hottest book
Quyển sách được mua nhiều nhất thường được mua chung với quyển sách nào

In [ ]:
top_books = df['original_title'].value_counts().head(10)
print(top_books)

original_title
The Fault in Our Stars         1524
The Martian                    1399
Divergent                      1342
Fangirl                        1333
Six of Crows                   1292
Scarlet                        1265
Daughter of Smoke & Bone       1225
A Court of Mist and Fury       1211
Gone Girl                      1171
A Court of Thorns and Roses    1159
Name: count, dtype: int64


In [ ]:
top_books.head()

,count
original_title,
The Fault in Our Stars,1524
The Martian,1399
Divergent,1342
Fangirl,1333
Six of Crows,1292


In [ ]:
print("Quyển sách được mua nhiều nhất: ", top_books.idxmax())

Quyển sách được mua nhiều nhất:  The Fault in Our Stars


In [ ]:
# xác định sách hot nhất
hot_book = str(top_books.idxmax())
print("Hot book:", hot_book)

# nới điều kiện luật
rules_hot = rules[
    rules['consequents'].apply(lambda x: hot_book in x)
]

# sắp xếp theo độ mạnh
rules_hot = rules_hot.sort_values(
    by=['confidence', 'lift'],
    ascending=False
)

rules_hot[['antecedents', 'consequents', 'support', 'confidence', 'lift']].head(10)

Hot book: The Fault in Our Stars


,antecedents,consequents,support,confidence,lift
291,(Looking for Alaska),(The Fault in Our Stars),0.013391,0.428850,4.665944
971,(Paper Towns),(The Fault in Our Stars),0.011504,0.394572,4.292996
719,(Delirium),(The Fault in Our Stars),0.011261,0.362745,3.946715
0,(Insurgent),(The Fault in Our Stars),0.018078,0.334083,3.634870
292,(Divergent),(The Fault in Our Stars),0.025504,0.313858,3.414813
894,(Legend),(The Fault in Our Stars),0.011504,0.304839,3.316686
37,(Allegiant),(The Fault in Our Stars),0.010408,0.301587,3.281310
787,(Eleanor & Park),(The Fault in Our Stars),0.019660,0.301306,3.278249
710,(Shatter Me),(The Fault in Our Stars),0.010043,0.289982,3.155047
86,(Mockingjay),(The Fault in Our Stars),0.014547,0.286915,3.121671


In [ ]:
rules_hot2 = rules[
    rules['antecedents'].apply(lambda x: hot_book in x)
]

rules_hot2 = rules_hot2.sort_values(
    by=['confidence', 'lift'],
    ascending=False
)

rules_hot2[['antecedents', 'consequents', 'support', 'confidence', 'lift']]

,antecedents,consequents,support,confidence,lift
293,(The Fault in Our Stars),(Divergent),0.025504,0.277483,3.414813
70,(The Fault in Our Stars),(Fangirl),0.020878,0.227152,2.820775
788,(The Fault in Our Stars),(Eleanor & Park),0.019660,0.213907,3.278249
856,(The Fault in Our Stars),(Gone Girl),0.018626,0.202649,2.848007


## Những quyển sách gây tranh cãi thường được mua kèm với quyển sách nào
Sách gây tranh cãi là những quyển sách có số rating cao và thấp ngang ngửa

In [ ]:
df_review1 = df_review1[df_review1['user_id'].isin(multi_rv_user['user_id'])]

In [ ]:
df_controversial = pd.merge(df_review1[['user_id', 'work_id', 'review_text', 'rating', 'started_at','read_at']], df_book[['work_id', 'original_title', 'genres', 'author', 'original_publication_year','avg_rating']],
              on = 'work_id', how = 'left')
df_controversial.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1139472 entries, 0 to 1139471
Data columns (total 11 columns):
 #   Column                     Non-Null Count    Dtype  
---  ------                     --------------    -----  
 0   user_id                    1139472 non-null  object 
 1   work_id                    1139472 non-null  int64  
 2   review_text                1139472 non-null  object 
 3   rating                     1103529 non-null  float64
 4   started_at                 793883 non-null   object 
 5   read_at                    1028286 non-null  object 
 6   original_title             1139472 non-null  object 
 7   genres                     1139472 non-null  object 
 8   author                     1139472 non-null  object 
 9   original_publication_year  1139133 non-null  float64
 10  avg_rating                 1139472 non-null  float64
dtypes: float64(3), int64(1), object(7)
memory usage: 95.6+ MB


In [ ]:
df_controversial = df_controversial[df_controversial['started_at'].notnull() & df_controversial['read_at'].notnull()]
df_controversial = df_controversial[df_controversial['started_at'] < df_controversial['read_at']]

Những quyển sách controversial thường bán tiếp được chung với nào:
- số rating <3 xấp xỉ số rating >3

In [ ]:
book_stats = (
    df_controversial
    .groupby('original_title')
    .agg(
        total_ratings=('rating', 'count'),
        pos_ratings=('rating', lambda x: (x > 3).sum()),
        neg_ratings=('rating', lambda x: (x <= 3).sum())
    )
)

In [ ]:
book_stats['pos_ratio'] = (
    book_stats['pos_ratings'] / book_stats['neg_ratings']
)

In [ ]:
controversial_books = book_stats[
    (book_stats['pos_ratio'] >= 0.7) &
    (book_stats['pos_ratio'] <= 1.3)
].sort_values(
    by='total_ratings',
    ascending=False
)

In [ ]:
controversial_books.head(10)

,total_ratings,pos_ratings,neg_ratings,pos_ratio
original_title,,,,
The Girl on the Train,1457,782,675,1.158519
Miss Peregrine's Home for Peculiar Children,1448,754,694,1.086455
"Harry Potter and the Cursed Child, Parts One and Two",1272,620,652,0.950920
City of Bones,1239,677,562,1.204626
The Maze Runner,1066,584,482,1.211618
The Selection,963,541,422,1.281991
Paper Towns,867,481,386,1.246114
Matched,691,323,368,0.877717
The Elite,649,318,331,0.960725


In [ ]:
# Những quyển sách gây tranh cãi thường được mua cùng với cuốn nào
contro_book = controversial_books['total_ratings'].idxmax()
print("Controversial book:", contro_book)

rules_contro = rules[
    rules['antecedents'].apply(lambda x: contro_book in x)
]

# sắp xếp theo độ mạnh
rules_contro = rules_contro.sort_values(
    by=['confidence', 'lift'],
    ascending=False
)

rules_contro[['antecedents', 'consequents', 'support', 'confidence', 'lift']].head(10)

Controversial book: The Girl on the Train


,antecedents,consequents,support,confidence,lift
433,(The Girl on the Train),(Gone Girl),0.014365,0.302953,4.257662
435,(The Girl on the Train),(The Martian),0.011504,0.242619,2.863494
434,(The Girl on the Train),(The Fault in Our Stars),0.010713,0.225931,2.458156


## Fanclub

In [ ]:
df_fc = df.copy()

df_fc['genres_list'] = df_fc['genres'].str.split(', ')

# Explode ra từng dòng. Mỗi dòng sẽ ứng với 1 thể loại đơn lẻ
df_fc = df_fc.explode('genres_list')

# Xóa khoảng trắng thừa (nếu có) để lọc cho chính xác
df_fc['genres_list'] = df_fc['genres_list'].str.strip()

print("Đã tạo xong cột 'genres_list'. Dữ liệu sẵn sàng!")
# Kiểm tra
display(df_fc[['user_id', 'original_title', 'genres_list']].head())

Đã tạo xong cột 'genres_list'. Dữ liệu sẵn sàng!


,user_id,original_title,genres_list
135566,631853097d378547c63cc2c72be75cd3,Ren Sheng gatokimekuPian dukenoMo Fa,non-fiction
135567,2f345f02ec439d1a25d8d319e9ca6748,Wool Omnibus,fiction
135567,2f345f02ec439d1a25d8d319e9ca6748,Wool Omnibus,fantasy
135567,2f345f02ec439d1a25d8d319e9ca6748,Wool Omnibus,paranormal
135567,2f345f02ec439d1a25d8d319e9ca6748,Wool Omnibus,mystery


In [ ]:
# Định nghĩa các nhóm Fanclub và thể loại tương ứng
fanclubs_config = {
    "Crime & Mystery": ['crime', 'mystery', 'thriller'],
    "Coming of age":   ['young-adult', 'fiction'],
    "Fantasy":         ['fantasy', 'paranormal']
}

# Cấu hình tham số cho mô hình (bạn có thể điều chỉnh tùy độ lớn dữ liệu)
MIN_SUPPORT = 0.03  # Tần suất xuất hiện tối thiểu (5%)
MIN_CONFIDENCE = 0.3 # Độ tin cậy tối thiểu (30%)

In [ ]:
def analyze_fanclub(df, club_name, target_genres):
    print(f"--- Đang xử lý: {club_name} ---")

    # BƯỚC 1: LỌC USER LÀ FAN CỨNG
    # Lấy các dòng có thể loại thuộc nhóm này
    mask_genre = df['genres_list'].isin(target_genres)

    # Đếm số sách unique mỗi user đã đọc trong thể loại này
    # Dùng nunique() vì dữ liệu đang bị duplicate do explode thể loại
    user_counts = df[mask_genre].groupby('user_id')['original_title'].nunique()

    # Lấy danh sách user đọc >= 3 cuốn
    valid_users = user_counts[user_counts >= 3].index.tolist()
    print(f"-> Số lượng thành viên: {len(valid_users)} người")

    if len(valid_users) == 0:
        print("-> Không đủ dữ liệu để chạy luật.")
        return None

    # BƯỚC 2: TẠO TRANSACTION (GIỎ SÁCH) CHO NHÓM NÀY
    # Lấy toàn bộ lịch sử đọc của các user này (để xem họ hay đọc cuốn gì đi kèm)
    df_club = df[df['user_id'].isin(valid_users)]

    # Gom nhóm: Mỗi user là 1 transaction, chứa danh sách tên sách (loại bỏ trùng lặp)
    transactions = df_club.groupby('user_id')['original_title'].apply(lambda x: list(set(x))).tolist()

    # BƯỚC 3: CHẠY FP-GROWTH
    # Mã hóa dữ liệu (One-hot encoding)
    te = TransactionEncoder()
    te_ary = te.fit(transactions).transform(transactions)
    df_trans = pd.DataFrame(te_ary, columns=te.columns_)

    # Tìm tập phổ biến (Frequent Itemsets)
    frequent_itemsets = fpgrowth(df_trans, min_support=MIN_SUPPORT, use_colnames=True)

    if frequent_itemsets.empty:
        print("-> Không tìm thấy tập phổ biến nào (hãy giảm min_support).")
        return None

    # Sinh luật kết hợp
    rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=MIN_CONFIDENCE)

    # Sắp xếp luật theo độ nâng (lift) để thấy các kết hợp thú vị nhất
    return rules.sort_values(by='lift', ascending=False)

In [ ]:
results = {}

for club, genres in fanclubs_config.items():
    # Gọi hàm xử lý
    rules_df = analyze_fanclub(df_fc, club, genres)

    if rules_df is not None and not rules_df.empty:
        # Lưu kết quả
        results[club] = rules_df

        # Hiển thị Top 5 luật mạnh nhất
        print(f"-> Top 5 luật kết hợp cho {club}:")
        display(rules_df[['antecedents', 'consequents', 'support', 'confidence', 'lift']].head(10))
    else:
        print(f"-> Không tìm thấy luật nào cho nhóm {club}")
    print("\n" + "="*50 + "\n")

--- Đang xử lý: Crime & Mystery ---
-> Số lượng thành viên: 11747 người
-> Top 5 luật kết hợp cho Crime & Mystery:


,antecedents,consequents,support,confidence,lift
53,(The Raven King),"(Blue Lily, Lily Blue)",0.031327,0.610282,10.480967
52,"(Blue Lily, Lily Blue)",(The Raven King),0.031327,0.538012,10.480967
50,"(Blue Lily, Lily Blue)",(The Dream Thieves),0.039840,0.684211,10.438209
51,(The Dream Thieves),"(Blue Lily, Lily Blue)",0.039840,0.607792,10.438209
64,(A Darker Shade of Magic),(A Gathering of Shadows),0.031583,0.417793,9.383962
65,(A Gathering of Shadows),(A Darker Shade of Magic),0.031583,0.709369,9.383962
3,(Clockwork Princess),(Clockwork Prince),0.032434,0.500657,9.305725
2,(Clockwork Prince),(Clockwork Princess),0.032434,0.602848,9.305725
55,(The Raven King),(The Dream Thieves),0.030561,0.595357,9.082667
54,(The Dream Thieves),(The Raven King),0.030561,0.466234,9.082667




--- Đang xử lý: Coming of age ---
-> Số lượng thành viên: 14202 người
-> Top 5 luật kết hợp cho Coming of age:


,antecedents,consequents,support,confidence,lift
20,"(Blue Lily, Lily Blue)",(The Dream Thieves),0.032953,0.683212,12.471687
21,(The Dream Thieves),"(Blue Lily, Lily Blue)",0.032953,0.601542,12.471687
24,(Queen of Shadows),(Heir of Fire),0.030911,0.606354,9.966937
25,(Heir of Fire),(Queen of Shadows),0.030911,0.508102,9.966937
22,(The Raven Boys),(The Dream Thieves),0.037248,0.481347,8.786742
23,(The Dream Thieves),(The Raven Boys),0.037248,0.679949,8.786742
12,(Crown of Midnight),(Heir of Fire),0.039431,0.527307,8.667608
13,(Heir of Fire),(Crown of Midnight),0.039431,0.648148,8.667608
30,(A Court of Mist and Fury),(A Court of Wings and Ruin),0.030841,0.369932,8.326118
31,(A Court of Wings and Ruin),(A Court of Mist and Fury),0.030841,0.694136,8.326118




--- Đang xử lý: Fantasy ---
-> Số lượng thành viên: 12357 người
-> Top 5 luật kết hợp cho Fantasy:


,antecedents,consequents,support,confidence,lift
46,"(Blue Lily, Lily Blue)",(The Dream Thieves),0.037873,0.686217,10.927298
47,(The Dream Thieves),"(Blue Lily, Lily Blue)",0.037873,0.603093,10.927298
3,(Clockwork Princess),(Clockwork Prince),0.030914,0.491634,9.552087
2,(Clockwork Prince),(Clockwork Princess),0.030914,0.600629,9.552087
57,(A Gathering of Shadows),(A Darker Shade of Magic),0.030509,0.696858,9.452327
56,(A Darker Shade of Magic),(A Gathering of Shadows),0.030509,0.413831,9.452327
55,(Heir of Fire),(Queen of Shadows),0.035526,0.508691,8.682168
54,(Queen of Shadows),(Heir of Fire),0.035526,0.606354,8.682168
28,(Cress),"(Winter, Scarlet)",0.031723,0.350626,8.665374
27,"(Winter, Scarlet)",(Cress),0.031723,0.784000,8.665374


## Tác giả

In [ ]:
#Encoding cho author
basket = (
    df[['user_id', 'author']]
    .dropna()
    .drop_duplicates()
    .groupby('user_id')['author']
    .apply(list)
    .reset_index()
)

basket.head()

te = TransactionEncoder()
te_array = te.fit(basket['author']).transform(basket['author'])

basket_encoded = pd.DataFrame(
    te_array,
    columns=te.columns_
)

basket_encoded.head()

,A. Lee Martinez,A. Meredith Walters,A. Zavarelli,A.A. Aguirre,A.A. Milne,A.B. Gayle,A.C. Bextor,A.C. Gaughen,A.D. Miller,A.E. Kirk,...,Zoe Dawson,Zoe Archer,Zoe Heller,Zoe Marriott,Zoe Sugg,Zoey Dean,Zora Neale Hurston,Zoraida Cordova,pleasefindthis,seventhswan
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False
4,False,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [ ]:
frequent_itemsets = fpgrowth(
    basket_encoded,
    min_support=0.03,
    use_colnames=True
)

rules = association_rules(
    frequent_itemsets,
    metric="confidence",
    min_threshold=0.3
)

rules = rules.sort_values('lift', ascending=False)
rules.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
138,(Amie Kaufman),"(Sarah J. Maas, Marissa Meyer)",0.085520,0.074502,0.031590,0.369395,4.958162,1.0,0.025219,1.467634,0.872968,0.245972,0.318631,0.396707
137,"(Sarah J. Maas, Marissa Meyer)",(Amie Kaufman),0.074502,0.085520,0.031590,0.424020,4.958162,1.0,0.025219,1.587694,0.862576,0.245972,0.370156,0.396707
141,"(Sarah J. Maas, Leigh Bardugo)",(Amie Kaufman),0.082172,0.085520,0.034451,0.419259,4.902498,1.0,0.027424,1.574679,0.867289,0.258566,0.364950,0.411053
142,(Amie Kaufman),"(Sarah J. Maas, Leigh Bardugo)",0.085520,0.082172,0.034451,0.402847,4.902498,1.0,0.027424,1.537007,0.870464,0.258566,0.349385,0.411053
194,"(Sarah J. Maas, V.E. Schwab)",(Leigh Bardugo),0.045712,0.144805,0.031225,0.683089,4.717307,1.0,0.024606,2.698536,0.825762,0.196026,0.629429,0.449363
